In [19]:
from datetime import timedelta, date
import datetime
import numpy as np
import pandas as pd
import string
import re
import csv
import requests
import string

df_remotes = pd.read_csv('turnstile_location.csv', delimiter = ',', header = 0, skipinitialspace = True)

df_remotes.drop('1', axis = 1, inplace = True)

In [22]:
print(df_remotes.head(5))

  Remote  Booth       Station Line Name Division
0   R001   A060  WHITEHALL ST        R1      BMT
1   R001   A058  WHITEHALL ST        R1      BMT
2   R001  R101S   SOUTH FERRY        R1      IRT
3   R002   A077     FULTON ST  ACJZ2345      BMT
4   R002   A081     FULTON ST  ACJZ2345      BMT


In [25]:
print(df_stations.head(5))

   Station ID  Complex ID GTFS Stop ID Division     Line  \
0           1           1          R01      BMT  Astoria   
1           2           2          R03      BMT  Astoria   
2           3           3          R04      BMT  Astoria   
3           4           4          R05      BMT  Astoria   
4           5           5          R06      BMT  Astoria   

                Stop Name Borough Daytime Routes Structure  GTFS Latitude  \
0  Astoria - Ditmars Blvd       Q            N W  Elevated      40.775036   
1            Astoria Blvd       Q            N W  Elevated      40.770258   
2                   30 Av       Q            N W  Elevated      40.766779   
3                Broadway       Q            N W  Elevated      40.761820   
4                   36 Av       Q            N W  Elevated      40.756804   

   GTFS Longitude  
0      -73.912034  
1      -73.917843  
2      -73.921479  
3      -73.925508  
4      -73.929575  


In [31]:
df_traffic = pd.read_csv('traffic_2017.csv', delimiter = ',', header = 0, skipinitialspace = True)

In [32]:
print(df_traffic.head(5))

    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  04/08/2017  00:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  04/08/2017  04:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  04/08/2017  08:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  04/08/2017  12:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  04/08/2017  16:00:00   

      DESC  ENTRIES    EXITS  
0  REGULAR  6127245  2076158  
1  REGULAR  6127285  2076165  
2  REGULAR  6127304  2076196  
3  REGULAR  6127409  2076284  
4  REGULAR  6127654  2076350  


In [37]:
df_traffic

<bound method DataFrame.info of            C/A  UNIT       SCP        STATION LINENAME DIVISION        DATE  \
0         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/08/2017   
1         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/08/2017   
2         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/08/2017   
3         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/08/2017   
4         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/08/2017   
5         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/08/2017   
6         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/09/2017   
7         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/09/2017   
8         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/09/2017   
9         A002  R051  02-00-00          59 ST  NQR456W      BMT  04/09/2017   
10        A002  R051  02-00-00          59 ST  NQR456W      BMT  04/09/2017   
11        A002  R051

In [38]:
df_traffic.describe()

,ENTRIES,EXITS
count,2.170121e+06,2.170121e+06
mean,3.781080e+07,3.016909e+07
std,2.032673e+08,1.820563e+08
min,0.000000e+00,0.000000e+00
25%,4.990290e+05,2.460920e+05
50%,2.636823e+06,1.486406e+06
75%,6.746402e+06,4.753406e+06
max,2.130545e+09,2.087387e+09


In [39]:
df_traffic.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [70]:
df_traffic_time = df_traffic.TIME.value_counts().sort_values(ascending = False)

print(df_traffic_time.head(10))

00:00:00    185204
04:00:00    185176
08:00:00    185070
20:00:00    185069
16:00:00    185052
12:00:00    184972
05:00:00    133307
01:00:00    133268
17:00:00    133208
09:00:00    133196
Name: TIME, dtype: int64


In [71]:
df_traffic_time.describe()

count     55139.000000
mean         39.357279
std        2380.645815
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max      185204.000000
Name: TIME, dtype: float64

'00:00:00'

In [60]:
df_traffic["DATE_TIME"] = pd.to_datetime(df_traffic.DATE + " " + df_traffic.TIME, format="%m/%d/%Y %H:%M:%S")

In [61]:
df_traffic

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,00:00:00,REGULAR,6127245,2076158,2017-04-08 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,04:00:00,REGULAR,6127285,2076165,2017-04-08 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,08:00:00,REGULAR,6127304,2076196,2017-04-08 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,12:00:00,REGULAR,6127409,2076284,2017-04-08 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,16:00:00,REGULAR,6127654,2076350,2017-04-08 16:00:00
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,20:00:00,REGULAR,6128002,2076398,2017-04-08 20:00:00
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/09/2017,00:00:00,REGULAR,6128172,2076417,2017-04-09 00:00:00
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/09/2017,04:00:00,REGULAR,6128213,2076419,2017-04-09 04:00:00
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/09/2017,08:00:00,REGULAR,6128226,2076440,2017-04-09 08:00:00
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/09/2017,12:00:00,REGULAR,6128293,2076512,2017-04-09 12:00:00
